In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
%pip install chess
%pip install svglib

In [2]:
import os
os.chdir('/content/drive/My Drive/cs6643final_project/')

In [3]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
import preprocess as pre


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
import cv2
import numpy as np
from collections import defaultdict

def segment_by_angle_kmeans(lines, k=2, **kwargs):

    default_criteria_type = cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER
    criteria = kwargs.get('criteria', (default_criteria_type, 10, 1.0))
    flags = kwargs.get('flags', cv2.KMEANS_RANDOM_CENTERS)
    attempts = kwargs.get('attempts', 10)

    angles = np.array([line[0][1] for line in lines])
    pts = np.array([[np.cos(2*angle), np.sin(2*angle)]
                    for angle in angles], dtype=np.float32)

    labels, centers = cv2.kmeans(pts, k, None, criteria, attempts, flags)[1:]
    labels = labels.reshape(-1)

    segmented = defaultdict(list)
    for i, line in enumerate(lines):
        segmented[labels[i]].append(line)
    segmented = list(segmented.values())
    return segmented

def intersection(line1, line2):
    rho1, theta1 = line1[0]
    rho2, theta2 = line2[0]
    A = np.array([
        [np.cos(theta1), np.sin(theta1)],
        [np.cos(theta2), np.sin(theta2)]
    ])
    b = np.array([[rho1], [rho2]])
    x0, y0 = np.linalg.solve(A, b)
    x0, y0 = int(np.round(x0)), int(np.round(y0))
    return [x0, y0]

def segmented_intersections(lines):
    intersections = []
    for i, group in enumerate(lines[:-1]):
        for next_group in lines[i+1:]:
            for line1 in group:
                for line2 in next_group:
                    intersections.append(intersection(line1, line2)) 

    return intersections

# 新段落

In [1]:
model_folder = '/content/drive/My Drive/cs6643final_project/'
model_file = "modified_vgg16.h5"
board_imgs = ["alpha_data_image45.jpeg"]
data_folder = "data/"
output_file_name = "board"
output_folder = "output/"

In [6]:
vgg = VGG16(weights="imagenet", include_top = False, input_shape= (224,224,3))
model = pre.model(vgg, model_file)

In [5]:
img = cv2.imread(data_folder + board_imgs[0])

In [8]:

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3,3), 0)
edges = cv2.Canny(blur,50,200,L2gradient=False)
lines = cv2.HoughLines(edges,1,np.pi/180,60)
for line in lines:
    for rho,theta in line:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),1)
segmented = segment_by_angle_kmeans(lines)
intersections = segmented_intersections(segmented)

In [ ]:
## sorted points by clustering
points = pre.group_points(intersections)
## get the average y in each clusters
f_points = pre.final_points(points)
## divide the board into small patches by the intersections
divide_the_board(img, points, "img/")

In [ ]:
## The path to the folder of images to be classified is "img/" (relative path), the * marker is for glob.glob() to use
## classify each cells on the board
labels = pre.classification(model, "img/*")
## transform labels to FEN notations
fen_notation = pre.result_to_fen(labels)

## draw the board with FEN notation
## The last parameter is the path to the output folder
pre.draw_board(fen_notation, output_file_name, )
output_img = cv2.imread(output_folder + output_file_name)
cv2.imshow(output_file_name, output_img)